# Abbreviations:

- `df` -> dataframe

- `Series` -> column / kolonne

- `embs` -> embeddings (numerical representation of words in a vector space, from our NLP AI model)

- `uembs` -> umap embeddings (dimensionality reduction of embs, to make it easier to visualize, 768 -> 2)

- `target_intersts` -> the 20 words we use as targets for our grouping of commit messages. We can se these, and we use them as targets for semi-supervision

- `group_topics` -> the topics we generate based on message cluster, less important for the ML pipeline, more for the human reader to understand what the clusters are about

# Imports

In [ ]:
from scripts import *

# Data

## Getting Data

In [ ]:
df = pd.read_csv("data/tensorflow-big.csv", on_bad_lines='skip')

len_start = len(df)

df = df[["author_name", "time_sec", "subject"]]


df = df.rename(columns={
    "author_name": "user", 
    "time_sec": "time_sec", 
    "subject": "text"
})

# dropping the two bots
df = df[df["user"] != "A. Unique TensorFlower"]
df = df[df["user"] != "TensorFlower Gardener"]

len_drop_bots = len(df)


# if your computer does not have GPU support, you can use a sample of the dataset instead to make it run in a reasonable time
if device == "cpu": df = df.sample(frac=0.05)

print(f"""
Rows before droppings bots:   {len_start}
Rows after dropping bots:     {len_drop_bots}
Rows diff:                    {len_start - len_drop_bots}
""")

df

We see that the commits are in order, and ready to be sliced timewise

In [ ]:
df = df.astype({"text" : str})

In [ ]:
# Consider Dropping A. Unique and Gardener

df["user"].value_counts().head(5)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

We are intending to use the `text` field as a temporary substitue of `categoryRaw` which we wait to get from the schibsted data

## Cleaning Data

In [ ]:
df["text_clean"] = df["text"].apply(string_cleaner)

df[["text", "text_clean"]].head(5)

## Engineering Data

### Making a new column for the week

In [ ]:
df["time_week"] = df["time_sec"].apply(lambda x: x//604800)

df.to_pickle(names["df"])

### Grouping on users

lag dictionary av å groupe på alle commit messages de har 
set groups på userId senere, så kan vi lage animation frames av hvordan grupper beveger seg

In [ ]:
# dfu - ABBR: Data Frame User grouped
dfu = df[["text_clean", "time_sec", "time_week", "user"]].copy()

dfu = dfu.groupby(["user", "time_week"]).agg(list).reset_index()

dfu["text_clean_join"] = dfu["text_clean"].apply(lambda x: " ".join(x))

dfu.head(3)

In [ ]:
weeks_per_user = dfu["user"].value_counts().reset_index()

print(len(weeks_per_user))

weeks_per_user[weeks_per_user["user"] > 10]

# Machine Learning

## Unsupervised ML

### NLP Embeddings

Getting the 768 dimensional embeddings for each commit message

In [ ]:
try:
    if conf["fresh_data"]: raise Exception
    embs = pickle.load(open(names[f"embs-{device}"], 'rb'))
    
except:
    embs = sbert_emb_getter(df["text_clean"].to_numpy(), filename=names[f"model-{device}"])
    pickle.dump(embs, open(names[f"embs-{device}"], 'wb'))

    conf["fresh_embs"] = True

print(f"fresh embs: {conf['fresh_embs']}")

### Dimensionality Reduction

We use UMAP to reduce the dimensionality of the embeddings from 768 to 2, so that we can visualize them

In [ ]:
umap_metric = "euclidean"

try:
    if conf["fresh_data"]: raise Exception
    uembs = pickle.load(open(names[f"uembs-{device}"], 'rb'))
    
except:
    uembs = UMAP(n_neighbors=15, min_dist=0.0).fit_transform(embs)
    pickle.dump(uembs, open(names[f"uembs-{device}"], 'wb'))

    conf["fresh_uembs"] = True

print(f"fresh uembs: {conf['fresh_uembs']}")

In [ ]:
# TODO make this plot just a trace, to fit in gridplot

fig = px.scatter(
  x=uembs[:,0], 
  y=uembs[:,1],
  range_x = [-25, 25],
  range_y = [-25, 25]
)

fig.update_layout(width=800, height=800)
fig.update_traces(marker=dict(size=2))

# plotting to show how the embeddings are when just dimensionality reduction is used
#fig_show_save(fig, "umap-scatter", show=conf["show_figs"])

fig.show()

In [ ]:
clusters_2d = HDBSCAN(min_cluster_size=100, min_samples=20, metric='euclidean', cluster_selection_method='eom').fit(uembs)


print(f"""
    2D
    Number of clusters: {len(set(clusters_2d.labels_)) - 1}
    Number of rows as outliers: {clusters_2d.labels_.tolist().count(-1)}
""")

## Semi Supervised

### Exploring Stopwords

#### Checking most common words

First checking without filtering for stopwords, then checking with filtering for stopwords

Then checking with filtering for english stopwords

In [ ]:
vc = (
    df["text_clean"].apply(lambda x: (x.split(" ")))
    .explode()
    .value_counts()
    .reset_index()
)

vc.head(10)

In [ ]:
# stopwords has been imported from nltk
s_words = stopwords.words('english')

print(f"""
    {type(s_words)}
    {len(s_words)}
    {s_words[0:10]}
""")

In [176]:
# vc where the index is not in the stopwords list
vc = vc[~vc["index"].isin(s_words)]

# getting the top 20 words
target_interest = list(vc.head(conf["interest_words"] + 1)["index"])

# removing the space that becomes the first element in the list
try:
  target_interest.remove("")
except:
  pass

print(target_interest)

names["target-interests"] = "data/target-interests.pkl"

pickle.dump(target_interest, open(names["target-interests"], 'wb'))


['add', 'fix', 'change', 'merge', 'update', 'test', 'remove', 'support', 'use', 'xla', 'build', 'request', 'pull', 'op', 'tests', 'tf', 'make', 'ops', 'error', 'api']


In [ ]:
# threshold for seeing if a commit message belongs to a interest or if not
threshold = 0.2

if not conf["generate_interests"]:
    target_interest = interest_fixer("""
    fix add merge remove update pull request python docs tensorflow generated
    """)

print(f"using generated target_interest: {conf['generate_interests']}")

In [ ]:
# custom stopwords as a union with the nltk stopwords and the target_interest found by value counting all the words in the commit messages
bonus_words = stopwords.words('english') + (target_interest)

# dumping for the plotting notebook
pickle.dump(bonus_words, open(names["bonus-words"], 'wb'))

In [ ]:
## TODO make this use the get embeddings function
# getting results.
y, similarity = make_dataset(embs, targets=target_interest, model=model, target_threshold=threshold)

In [ ]:
print(y[5])

In [ ]:
set(y)

list(y).index(0)

y[4]

In [ ]:
set(target_interest)

In [ ]:

# liste av indexes
target_interest_list = [target_interest[i-1] for i in y]

print(len(target_interest_list) - len(df))

print(target_interest_list[0:10])

names["target-interest-list"] = "data/target-interest-list.pkl"

pickle.dump(target_interest_list, open(names["target-interest-list"], 'wb'))

In [ ]:
set(target_interest_list)
len(set(target_interest_list))

In [ ]:
try:
  if conf["fresh_data"]: raise Exception

  uemb_semi_s = pickle.load( open( names[f"uembs-s-{device}"], "rb" ) )
  
except:
  # used to have just nn = 100 at 0.2 similiarity, and not the metric and target weight
  # target weight is between 0 - 1, 0.5 is default, we used 1 for a while
  uemb_semi_s = UMAP(n_neighbors=15, min_dist=0.0, target_weight=0.5).fit_transform(embs, y-1)
  pickle.dump( uemb_semi_s, open( names[f"uembs-s-{device}"], "wb" ) )

  conf["fresh_s_uembs"] = True

print(f"fresh semi supervised uembs: {conf['fresh_s_uembs']}")

In [ ]:
cluster_semi_s_hdb = HDBSCAN(min_cluster_size=100, min_samples=20, metric='euclidean', cluster_selection_method='eom').fit(uemb_semi_s)

## Cluster & Topic Inspection

In [ ]:
result_2d = result_df_maker(uembs, clusters_2d.labels_, df["text_clean"].to_numpy(), bonus_words=target_interest)

vcr = result_2d[["cluster_label", "group_topics"]].groupby(["cluster_label", "group_topics"])["group_topics"].count().reset_index(name="commit_count").sort_values(by="commit_count", ascending=False).head(20)

vcr

In [ ]:
result_2d_semi = result_df_maker(uemb_semi_s, y, df["text_clean"].to_numpy(), bonus_words=target_interest)

vcr = result_2d_semi[["cluster_label", "group_topics"]].groupby(["cluster_label", "group_topics"])["group_topics"].count().reset_index(name="commit_count").sort_values(by="commit_count", ascending=False).head(20)

vcr

# cluster labelis here interest label

In [ ]:
# finding most common words in 20 most common group_topics to see if we need more stopwords
vcr["group_topics"].apply(lambda x: x.split(" ")).explode().value_counts().head(20)

# Making Result DF

Kan gjøre regersjons øking per interesse, og velge hvilken som er mest likely ved et tidspunkt

In [ ]:
dfres = df[["text_clean", "time_sec", "time_week", "user"]].copy()

#dfres["time_week"] = dfres["time_sec"].apply(lambda x: datetime.fromtimestamp(x).isocalendar()[1])

#dfres["time_week"] = dfres["time_sec"].apply(lambda x: x//604800)

dfres["x"] = uemb_semi_s[:, 0]

dfres["y"] = uemb_semi_s[:, 1]

dfres["cluster"] = cluster_semi_s_hdb.labels_



dfres["interest_id"] = list(y)

# -1 to make up for adding 1 earlier
dfres["target_interest"] = dfres["interest_id"].apply(lambda x: target_interest[x-1])

# find topic by interest instead
topic_dict = topic_by_clusterId(dfres["text_clean"].to_numpy(), dfres["interest_id"].to_numpy(), bonus_words=bonus_words)

dfres["topic"] = dfres["interest_id"].apply(lambda x: " ".join(list(topic_dict[x])))

dfres = dfres[dfres["cluster"] != -1]

# Pickling the dfres for plotting in other notebook
dfres.to_pickle(names["dfres"])

dfres

# Data for Exploration

### Data Restructuring

- Grouping by user to get info on their commits and which target_interest their commits belong to in a quantitative way
- Using the user groups, we can again group the df by user groups and time and now have very few groups, and we can do regression on their activity over time

In [173]:
userdf = pd.DataFrame({
    "user": df["user"],
#    "time_week" : list(df["time_week"]),
    "target_interest_id" : list(y),
    "cluster_id" : list(cluster_semi_s_hdb.labels_)
    })


userdf["target_interest"] = userdf["target_interest_id"].apply(lambda x: target_interest[x-1])


userdf = userdf.groupby("user").agg(list).reset_index()


print(len(userdf))

userdf.to_pickle(names["df-user"])

3803


In [ ]:
usergroupdf = pd.DataFrame({
    "user": df["user"],
    "time_sec" : list(df["time_sec"]),
    "target_interest_id" : list(y),
    })

# mapping in the target interest
usergroupdf["target_interest"] = usergroupdf["target_interest_id"].apply(lambda x: target_interest[x-1])

# Setting cluster id on the users to get the cluster id for each user
usergroupdf["user_group_id"] = usergroupdf["user"].apply(lambda x: userdId_groupID_dict[x])

# Making time sec into time day
usergroupdf["time_day"] = usergroupdf["time_sec"].apply(lambda x: x//(60*60*24))

usergroupdf.sample(5)

names["user-group-df"] = "data/df-user-group.pkl"

usergroupdf.to_pickle(names["df-user-group"])